## Trabajo 02 Renta fija Sql y python - Laura Rojas - Edwin Rios

#### Librerias

In [1]:
import mysql.connector
import datetime as dt
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


In [2]:
import os
import sys
import inspect

In [3]:
import Clase_6.Funciones as funciones
from Clase_6.Funciones import DB_class
import Clase_6.Clase_6Fun as clase_6Fun

--funciones\
    |-DB_class\
    |-coupon_m(parametros)\
    |-discount_factor_m(parametros)\
    |-dirty_price_m(parametros)\
    |-dirty_price_1i(parametros,inst_pos)\
    |-ytm_1d_obj_func(x,i,params,prices)\
    |-find_ytm_loop(parametros,prices)\
    |-ytm_obj_func(x,params,prices)\
    |-find_ytm_m(parametros,prices)\
    |-NelsonSiegel_1d(params, ven)\
    |-NelsonSiegel_m(params, ven)\
    |-dirty_price_m_ns(betas,parametros)\
    |-objetive_function(betas,parametros)\
    |-betas_optimization(parametros)\
    |-betas_optimization_bound(parametros)\

--Clase_6Fun\
    |-Macaulay_duration_mat(parametros,prices)\
    |-Modified_duration_mat(parametros,prices)\
    |-Modified_duration_mat_2(parametros,prices,macaulay_duration)\
    |-Convexity_mat(parametros,prices)\
    |-Macaulay_duration_mat_loop(parametros,prices)\
    |-Modified_duration_mat_loop(parametros,prices)\
    |-Modified_duration_mat_loop_2(parametros,prices,macaulay_duration)\
    |-Convexity_mat_loop(parametros,prices)\
    |-objetive_function_prices(betas,parametros)\
    |-betas_optimization_bound_prices(parametros)\

In [4]:
import itertools
import importlib

#### Inclusión desde base de datos

In [128]:
db_object = DB_class(host="localhost",user="root",password="root",database="trabajo_1")
class2_db = db_object.open_db()
mi_cursor = class2_db.cursor()
query = ("SELECT * FROM trabajo_2.tabla_precios WHERE trabajo_2.tabla_precios.frecuencia_pago=6 "+\
    "order by trabajo_2.tabla_precios.Tipo, trabajo_2.tabla_precios.Indice_tipo limit 40")
tes_faciales = pd.read_sql(query, con=class2_db)
display(tes_faciales)

C:\Users\Edwin\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Tipo,Indice_tipo,fecha_valoracion,nemotecnico,fecha_emision,fecha_vencimiento,cupon,tir,frecuencia_pago,Indicador_indexado,conteo_dias,precio_sucio,duracion_mac,duracion_mod,Convexidad
0,T1,1,2022-09-18,CDTOCSS6V,2012-04-12,2023-04-12,6.00,4.021916,6,No,REAL/365,0.0,0.0,0.0,0.0
1,T1,2,2022-09-18,CDTGEDS6V,2012-08-26,2023-08-26,6.00,4.760056,6,No,REAL/365,0.0,0.0,0.0,0.0
2,T1,3,2022-09-18,CDTDXAS6V,2012-05-15,2024-05-15,6.00,5.905344,6,No,REAL/365,0.0,0.0,0.0,0.0
3,T1,4,2022-09-18,CDTVFTS6V,2012-07-24,2024-07-24,7.00,6.157886,6,No,REAL/365,0.0,0.0,0.0,0.0
4,T1,5,2022-09-18,CDTAOFS6V,2014-12-20,2024-12-20,5.00,6.634711,6,No,REAL/365,0.0,0.0,0.0,0.0
5,T1,6,2022-09-18,CDTSESS6V,2012-03-12,2025-03-12,6.00,6.865697,6,No,REAL/365,0.0,0.0,0.0,0.0
6,T1,7,2022-09-18,CDTXWMS6V,2016-10-11,2025-10-11,5.00,7.377591,6,No,REAL/365,0.0,0.0,0.0,0.0
7,T1,8,2022-09-18,CDTRZLS6V,2015-11-26,2025-11-26,5.00,7.473473,6,No,REAL/365,0.0,0.0,0.0,0.0
8,T1,9,2022-09-18,CDTCUIS6V,2014-01-20,2026-01-20,7.00,7.582053,6,No,REAL/365,0.0,0.0,0.0,0.0
9,T1,10,2022-09-18,CDTRKZS6V,2012-08-26,2026-08-26,6.00,7.954766,6,No,REAL/365,0.0,0.0,0.0,0.0


In [130]:
val_date= tes_faciales.fecha_valoracion.drop_duplicates()[0]
ini_date = tes_faciales.fecha_emision
fin_date = tes_faciales.fecha_vencimiento
cpn =  tes_faciales.cupon
discount_rates = tes_faciales.tir
frec_pag=tes_faciales.frecuencia_pago

In [131]:
### titulos
mul_inst = 1 #intentar 1, 100 y 10000
parametros = dict({
        'val_date':val_date, 
        'ini_date':np.tile(ini_date,mul_inst), 
        'fin_date':np.tile(fin_date,mul_inst), 
        'coupon':np.tile(cpn,mul_inst),
        'discount_rates':np.tile(discount_rates,mul_inst),
        'frec_pag':np.tile(frec_pag,mul_inst),
        'limits':np.array([-20,20])
        })

In [90]:
ini_date.shape

(200,)

In [132]:
importlib.reload(funciones)

<module 'Clase_6.Funciones' from 'd:\\cursos\\Maestria finanzas\\S3_PySql rent fij\\Trabajo02\\Clase_6\\Funciones.py'>

In [133]:
n_inst = len(parametros['coupon'])
## Funcion de matriz de cupones
coupon_dict = funciones.coupon_m6(parametros)
parametros['cpn_dict'] = coupon_dict
parametros['delta_t'] = coupon_dict['days_matrix']/365
parametros['discount_factors'] = funciones.discount_factor_m(parametros)

prices = funciones.dirty_price_m(parametros)

(24,)
(24,)
(26,)
(26,)
(22,)
(28,)
(20,)
(22,)
(26,)
(30,)
(20,)
(34,)
(16,)
(18,)
(22,)
(16,)
(34,)
(22,)
(28,)
(18,)
(24,)
(22,)
(42,)
(42,)
(36,)
(30,)
(22,)
(26,)
(20,)
(28,)
(40,)
(36,)
(34,)
(30,)
(36,)
(32,)
(32,)
(38,)
(46,)
(34,)


In [27]:
x_0=0
for i in range(10):
    x_0=+i*12
    print(x_0)

0
12
24
36
48
60
72
84
96
108


In [120]:
tes_faciales.fecha_emision[0]
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()
periodos=10
frecuencia_0=3
years_0=0
months_0=0
arr = np.array([])
for i in range(periodos):
    if i==0:
        years_0=0
    elif (frecuencia_0*i)%12==0:
        years_0=(frecuencia_0*i)/12
    months_0=(frecuencia_0*i)%12
    
    respuesta=issue + relativedelta(years=years_0) +relativedelta(months=months_0) 
    arr = np.append(arr, respuesta)    
    #print(years_0,' - ',months_0,' - ',respuesta)
print(arr)

[datetime.date(2012, 9, 11) datetime.date(2012, 12, 11)
 datetime.date(2013, 3, 11) datetime.date(2013, 6, 11)
 datetime.date(2013, 9, 11) datetime.date(2013, 12, 11)
 datetime.date(2014, 3, 11) datetime.date(2014, 6, 11)
 datetime.date(2014, 9, 11) datetime.date(2014, 12, 11)]


In [121]:
arr.shape

(10,)

In [87]:
tes_faciales.fecha_emision[0]
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()
periodos=10
for i in range(periodos):
    print(issue + relativedelta(years=+i))

2012-04-12
2013-04-12
2014-04-12
2015-04-12
2016-04-12
2017-04-12
2018-04-12
2019-04-12
2020-04-12
2021-04-12


In [78]:
tes_faciales.fecha_emision[0]

datetime.date(2012, 4, 12)

In [79]:
issue=dt.datetime.strptime(str(tes_faciales.fecha_emision[0]),'%Y-%m-%d').date()

In [80]:
#respuesta=issue + relativedelta(months=+1) 
respuesta=issue + relativedelta(years=0) +relativedelta(months=6) 
print(respuesta)

2012-10-12


In [ ]:
coupon_dates = np.array([issue + relativedelta(years=1,month=0) for x in range(int(flow))])

In [71]:
from datetime import datetime

In [76]:
now = datetime.now()
now = dt.datetime.strptime(str(datetime.now()),'%Y-%m-%d').date()
print(now)  # 👉️ 2022-06-22 19:27:37.483564

result = now + relativedelta(months=+3)

print(result)  # 👉️ 2022-09-22 19:27:37.483564

ValueError: unconverted data remains:  23:18:22.913338

In [112]:
prueba=np.array([1,2,2])

In [113]:
prueba.shape

(3,)

In [116]:
 
# creating an array
arr = np.array([1, 8, 3, 3, 5])
print('Original Array : ', arr)
 
# appending to the array
arr = np.append(arr, [7])
print('Array after appending : ', arr)

Original Array :  [1 8 3 3 5]
Array after appending :  [1 8 3 3 5 7]


In [117]:
arr.shape

(6,)

In [118]:
# creating an array
arr = np.array([])
print('Original Array : ', arr)
 
# appending to the array
arr = np.append(arr, [7])
print('Array after appending : ', arr)

Original Array :  []
Array after appending :  [7.]
